In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Loading data
file = ('Movies.csv')
movies_df = pd.read_csv(file)
movies_df = movies_df.dropna()
# movies_df['IMDb'] = movies_df['IMDb'].loc(: movies_df[''])
movies_df.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [3]:
movies_df.loc[movies_df['IMDb'] >= 8, 'Rating Category'] = 'Good Movie'
movies_df.loc[(movies_df['IMDb'] >= 5) & (movies_df['IMDb'] < 8), 'Rating Category'] = 'Average Movie'
movies_df.loc[movies_df['IMDb'] < 5, 'Rating Category'] = 'Bad'

In [4]:
le = LabelEncoder()
df2 = movies_df.copy()
# df2['']
df2 = pd.get_dummies(df2, columns=['Age'])
# df2['Directors'] = le.fit_transform(df2['Directors'].astype(str))
# df2['Genres'] = le.fit_transform(df2['Genres'].astype(str))
# df2['Country'] = le.fit_transform(df2['Country'].astype(str))
df2['Language'] = le.fit_transform(df2['Language'].astype(str))
df2['Rating Category'] = le.fit_transform(df2['Rating Category'].astype(str))
df2.head()

,Unnamed: 0,ID,Title,Year,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,...,Genres,Country,Language,Runtime,Rating Category,Age_13+,Age_16+,Age_18+,Age_7+,Age_all
0,0,1,Inception,2010,8.8,87%,1,0,0,0,...,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom",163,148.0,2,1,0,0,0,0
1,1,2,The Matrix,1999,8.7,87%,1,0,0,0,...,"Action,Sci-Fi",United States,32,136.0,2,0,0,1,0,0
2,2,3,Avengers: Infinity War,2018,8.5,84%,1,0,0,0,...,"Action,Adventure,Sci-Fi",United States,32,149.0,2,1,0,0,0,0
3,3,4,Back to the Future,1985,8.5,96%,1,0,0,0,...,"Adventure,Comedy,Sci-Fi",United States,32,116.0,2,0,0,0,1,0
4,4,5,"The Good, the Bad and the Ugly",1966,8.8,97%,1,0,1,0,...,Western,"Italy,Spain,West Germany",347,161.0,2,0,0,1,0,0


In [5]:
df2

,Unnamed: 0,ID,Title,Year,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,...,Genres,Country,Language,Runtime,Rating Category,Age_13+,Age_16+,Age_18+,Age_7+,Age_all
0,0,1,Inception,2010,8.8,87%,1,0,0,0,...,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom",163,148.0,2,1,0,0,0,0
1,1,2,The Matrix,1999,8.7,87%,1,0,0,0,...,"Action,Sci-Fi",United States,32,136.0,2,0,0,1,0,0
2,2,3,Avengers: Infinity War,2018,8.5,84%,1,0,0,0,...,"Action,Adventure,Sci-Fi",United States,32,149.0,2,1,0,0,0,0
3,3,4,Back to the Future,1985,8.5,96%,1,0,0,0,...,"Adventure,Comedy,Sci-Fi",United States,32,116.0,2,0,0,0,1,0
4,4,5,"The Good, the Bad and the Ugly",1966,8.8,97%,1,0,1,0,...,Western,"Italy,Spain,West Germany",347,161.0,2,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16657,16657,16658,A Kid in King Arthur's Court,1995,4.7,5%,0,0,0,1,...,"Adventure,Comedy,Family,Fantasy,Romance","United States,Hungary,United Kingdom",32,89.0,1,0,0,0,1,0
16671,16671,16672,George of the Jungle 2,2003,3.3,17%,0,0,0,1,...,"Adventure,Comedy,Family","United States,Australia",32,87.0,1,0,0,0,1,0
16677,16677,16678,That Darn Cat,1997,4.7,13%,0,0,0,1,...,"Comedy,Crime,Family,Thriller",United States,74,116.0,1,0,0,0,1,0
16687,16687,16688,Kazaam,1996,3.0,6%,0,0,0,1,...,"Comedy,Family,Fantasy,Musical",United States,32,93.0,1,0,0,0,1,0


In [6]:
# Set features and target

X = df2.copy()
# X = X.drop(['Unnamed: 0', 'ID', 'Title', 'Rotten Tomatoes', 'IMDb'], axis=1)
X = X.drop(['Unnamed: 0', 'ID', 'Title', 'Rotten Tomatoes', 'IMDb', 'Directors', 'Country', 'Genres'], axis=1)

display(X.head())

y = df2['Rating Category'].astype(str).ravel()
y

,Year,Netflix,Hulu,Prime Video,Disney+,Type,Language,Runtime,Rating Category,Age_13+,Age_16+,Age_18+,Age_7+,Age_all
0,2010,1,0,0,0,0,163,148.0,2,1,0,0,0,0
1,1999,1,0,0,0,0,32,136.0,2,0,0,1,0,0
2,2018,1,0,0,0,0,32,149.0,2,1,0,0,0,0
3,1985,1,0,0,0,0,32,116.0,2,0,0,0,1,0
4,1966,1,0,1,0,0,347,161.0,2,0,0,1,0,0


array(['2', '2', '2', ..., '1', '1', '1'], dtype=object)

In [7]:
# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fitting the data
X_scaler = scaler.fit(X_train)

# Scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=1)
rf_model

RandomForestClassifier(n_estimators=500, random_state=1)

In [10]:
rf_model = rf_model.fit(X_train_scaled, y_train)
rf_model

RandomForestClassifier(n_estimators=500, random_state=1)